Lidando com diferentes layout de sites
O código abaixo, mostra um exemplo de uma classe "Content" e duas funções de coleta de dados que aceitam um objeto BeautifulSoup e devolvem uma instância de Content.

In [1]:
import requests
from bs4 import BeautifulSoup

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

def scrapeNoticiasUol(url):
    bs = getPage(url)
    title = bs.find("h1").text
    lines = bs.find_all("p")
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = getPage(url)
    title = bs.find("h1").text
    body = bs.find("div", {"class", "post-body"}).text
    return Content(url, title, body)

""" url = 'https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/'
content = scrapeBrookings(url)
print('Title: {}'.format(content.title))
print('URL: {}'.format(content.url))
print(content.body) """


# Obs: No livro o autor usou o site do The New York Times, porém agora o site
# Solicita que o leitor adquira um plano de leitura.
# Para trazer algo que funcione ao código adaptei o código para o Uol Notícias
url = 'https://noticias.uol.com.br/ultimas-noticias/ansa/2022/08/08/comprador-rejeita-1-carga-de-graos-que-saiu-da-ucrania.htm'
content = scrapeNoticiasUol(url)
print('Title: {}'.format(content.title))
print('URL: {}'.format(content.url))
print(content.body)

Title:   1º navio de grãos liberado após acordo entre Rússia e Ucrânia é rejeitado por atraso   
URL: https://noticias.uol.com.br/ultimas-noticias/ansa/2022/08/08/comprador-rejeita-1-carga-de-graos-que-saiu-da-ucrania.htm
Notícias do conflito entre Rússia e Ucrânia
08/08/2022 18h15 
A primeira carga de grãos de milho que saiu da Ucrânia após o acordo com a Rússia para o desbloqueio do Mar Negro foi rejeitada pelo comprador e agora aguarda um novo interessado na compra, informou a embaixada do país em Beirute nesta segunda-feira (8).
As 26 mil toneladas do produto deveriam ter sido desembarcadas neste domingo (7), mas o navio Razoni ainda continua próximo ao mar turco. Por conta do adiamento, a embarcação não pode entrar no porto de Beirute, no Líbano.

O comprador teria desistido do produto pelo atraso na entrega, de mais de cinco meses. Agora, a embaixada informou que está em busca de um novo comprador - que pode estar no Líbano ou em algum outro país da região.
O bloqueio aos portos 

Desenvolvendo um Crawler para coletar o título e informações do conteúdo de qualquer site, utilizando 2 classes que pode poupar códigos repetidos.

In [ ]:

class Content:
    """
        Classe base comum para todos os artigos/páginas
    """
    
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        """
            Uma função flexível de exibição controla a saída
        """
        print("URL: {}".format(self.url))
        print("TITLE: {}".format(self.title))
        print("BODY: {}".format(self.body))
            
class Website:
    """
        Contém informações sobre a estrutura do site
    """
    
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        
# As classes acima servirão de base para de fato eu construir o crawler utilizando agora as 2 classes criadas acima
        
import requests
from bs4 import BeautifulSoup

class Crawler:
    
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    def sageGet(self, pageObj, selector):
        """
            Função utilitária usada para obter uma string de conteúdo de um objeto BeautifulSoup e um seletor.
            Devolve uma string vazia caso nenhum objeto seja encontrado para o dado seletor
        """
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        
        return ''
    
    def parse(self, site, url):
        """
            Extrai conteúdo de um dado URL de página
        """
        
        bs = self.getPage(url)
        if bs is not None:
            title = self.sageGet(bs, site.titleTag)
            body = self.sageGet(bs, site.bodyTag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()


# Abaixo o código que instância os objetos criados acima, e dar início ao processo

crawler = Crawler()

siteData = [
    ['O\'Reilly Media', 'http://oreilly.com', 'h1', 'section#product-description'],
    ['Reuters', 'http://reuters.com', 'h1', 'div.StandardArticleBody_body_1gnLA'],
    ['Brookings', 'http://www.brookings.edu', 'h1', 'div.post-body']
]

websites = []
for row in siteData:
    websites.append(Website(row[0],row[1],row[2],row[3]))
    

crawler.parse(websites[0], 'http://shop.oreilly.com/product/0636920028154.do')
crawler.parse(websites[1], 'http://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0')
crawler.parse(websites[2], 'http://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/')

        